# Vertex AI and LLamaIndex Knowledge Graph RAG Query Engine


## Graph RAG

Graph RAG is an Knowledge-enabled RAG approach to retrieve information from Knowledge Graph on given task. Typically, this is to build context based on entities' SubGraph related to the task.

## NebulaGraph Property Graph Index
NebulaGraph is an open-source distributed graph database built for super large-scale graphs with milliseconds of latency.

If you already have an existing graph, please skip to the end of this notebook.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

```python
%pip install llama-index-graph-stores-nebula
%pip install llama-index-llms-vertex
%pip install llama-index-embeddings-vertex
%pip install llama-index
```


Before we start the `Knowledge Graph RAG QueryEngine` demo, let's first get ready for basic preparation of Llama Index.

In [1]:
# For Vertex

import os


import logging
import google.auth
import google.auth.transport.requests
from google.cloud import aiplatform
import vertexai
from vertexai.generative_models import HarmBlockThreshold, HarmCategory, SafetySetting
import sys
from dotenv import load_dotenv

load_dotenv()  # this loads the .env script for use below
PROJECT_ID = os.getenv("PROJECT_ID")
LOCATION = os.getenv("LOCATION")
NEBULA_SERVER_ADDRESS = "127.0.0.1"

credentials = google.auth.default(quota_project_id=PROJECT_ID)[0]
request = google.auth.transport.requests.Request()
credentials.refresh(request)

safety_config = [
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=HarmBlockThreshold.BLOCK_NONE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=HarmBlockThreshold.BLOCK_NONE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=HarmBlockThreshold.BLOCK_NONE,
    ),
]

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Define the LLM


In [2]:
from llama_index.llms.vertex import Vertex
from llama_index.embeddings.vertex import VertexTextEmbedding
from llama_index.core import Settings

Settings.llm = Vertex(
    temperature=0,
    model="gemini-1.5-flash",
    credentials=credentials,
    safety_settings=safety_config,
)
Settings.embed_model = VertexTextEmbedding(
    model_name="text-embedding-004", credentials=credentials
)
Settings.chunk_size = 256

# Nested Asyncio needed for async ops

In [3]:
import nest_asyncio

nest_asyncio.apply()

## Prepare for NebulaGraph

In [4]:
space_name = "gog3"

Then we could instiatate a `NebulaGraphStore`, in order to create a `StorageContext`'s `graph_store` as it.

In [5]:
%load_ext ngql
%ngql --address $NEBULA_SERVER_ADDRESS --port 9669 --user root --password <password>
%ngql CREATE SPACE IF NOT EXISTS $space_name(vid_type=FIXED_STRING(256))

[OK] Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


""


# **One time Operation to Load the DB**

In [6]:
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaPropertyGraphStore

graph_store = NebulaPropertyGraphStore(space=space_name, overwrite=True)

Here, we assumed to have the same Knowledge Graph from [this turtorial](https://gpt-index.readthedocs.io/en/latest/examples/query_engine/knowledge_graph_query_engine.html#optional-build-the-knowledge-graph-with-llamaindex)

Let's follow on this tutorial:

# With the help of Llama Index and LLM defined, we could build Knowledge Graph from given documents.

If we have a Knowledge Graph on NebulaGraphStore already, this step could be skipped

Load data from Wikipedia for "Guardians of the Galaxy Vol. 3"

In [7]:
from llama_index.core import download_loader

from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()

documents = loader.load_data(
    pages=["Guardians of the Galaxy Vol. 3"], auto_suggest=False
)

# Next, Generate a KnowledgeGraphIndex with NebulaGraph as graph_store
Then, we will create a KnowledgeGraphIndex to enable Graph based RAG, apart from that, we have a Knowledge Graph up and running for other purposes, too!

## Create a vector store

In [8]:
from llama_index.core.vector_stores.simple import SimpleVectorStore

vec_store = SimpleVectorStore()

In [9]:
from llama_index.core.indices.property_graph import PropertyGraphIndex

index = PropertyGraphIndex.from_documents(
    documents,
    property_graph_store=graph_store,
    vector_store=vec_store,
    show_progress=True,
)

index.storage_context.vector_store.persist("./data/nebula_vec_store.json")

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 351/351 [00:01<00:00, 189.20it/s]


# Loading from an existing graph
**Important** Go here if skipping data creation step above

In [6]:
from llama_index.graph_stores.nebula import NebulaPropertyGraphStore
from llama_index.core.indices.property_graph import PropertyGraphIndex


graph_store = NebulaPropertyGraphStore(space=space_name)

from llama_index.core.vector_stores.simple import SimpleVectorStore

vec_store = SimpleVectorStore.from_persist_path("./data/nebula_vec_store.json")

index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store,
    vector_store=vec_store,
)

### Now that the graph is created, we can explore it with [jupyter-nebulagraph](https://github.com/wey-gu/jupyter_nebulagraph)

In [7]:
# Query some random Relationships with Cypher

%ngql USE $space_name;
%ngql MATCH (p)-[e]->(q) RETURN e LIMIT 10

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,e
0,"(""dd28a48b-ccee-405f-bd46-e1e8e2434694"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""PREVIOUS"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""dd28a48b-ccee-405f-bd46-e1e8e2434694""}]->(""5285dbae-8bec-442f-a1c3-a36c33b5104a"")"
1,"(""dd28a48b-ccee-405f-bd46-e1e8e2434694"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""NEXT"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""dd28a48b-ccee-405f-bd46-e1e8e2434694""}]->(""70d46625-3e9b-4143-8134-ed14841085a7"")"
2,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Released in"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""8b484da2-9696-4354-8062-e3785041f5ea""}]->(""2023"")"
3,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Reveals"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""b1571283-4451-4034-9fbf-2f39d97f88d8""}]->(""Backstory"")"
4,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Fulfills"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""093be039-becb-45f4-b158-1c59a0b464c8""}]->(""Character arc"")"
5,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Is"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""324f89e8-afee-4149-a370-c8c9e0fcc7bd""}]->(""Emotional"")"
6,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Put on hold"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""bcbebca2-ac32-4fca-90d4-dfb371be708d""}]->(""February 2021"")"
7,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Filming"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""fe695498-a8b6-46e1-82ee-84f7e828d7d9""}]->(""February to late april 2022"")"
8,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Is"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""30317f7b-94fe-4bef-9386-fc28f4a82bc3""}]->(""First mcu film"")"
9,"(""Vol. 3"")-[:Relation__@0{_node_content: __NULL__, _node_type: __NULL__, creation_date: __NULL__, doc_id: __NULL__, document_id: __NULL__, file_name: __NULL__, file_path: __NULL__, file_size: __NULL__, file_type: __NULL__, label: ""Uses"", last_modified_date: __NULL__, ref_doc_id: __NULL__, triplet_source_id: ""80d32c35-3720-404f-8ae7-9b3ba2eb5090""}]->(""Flashbacks"")"


In [56]:
TAGS = %ngql SHOW TAGS
%ngql SHOW TAGS

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,Chunk__
1,Entity__
2,Node__
3,Props__


In [90]:
EDGES = %ngql SHOW EDGES
%ngql SHOW EDGES

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,Relation__
1,__meta__node_label__
2,__meta__rel_label__


In [ ]:
# Index the schema for querying
%ngql CREATE TAG INDEX IF NOT EXISTS entity_index on Entity__();
%ngql CREATE TAG INDEX IF NOT EXISTS props_index on Props__();
%ngql CREATE EDGE INDEX IF NOT EXISTS relation_index on Relation__();

In [103]:
%ngql MATCH p=(v:Entity__)-[r]->(t:Entity__) RETURN v.Entity__.name AS src, r.label AS relation, t.Entity__.name AS dest LIMIT 15;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,src,relation,dest
0,Vol. 3,Released in,2023
1,Vol. 3,Reveals,Backstory
2,Vol. 3,Fulfills,Character arc
3,Vol. 3,Is,Emotional
4,Vol. 3,Put on hold,February 2021
5,Vol. 3,Filming,February to late april 2022
6,Vol. 3,Is,First mcu film
7,Vol. 3,Uses,Flashbacks
8,Vol. 3,Tonal difference,Holiday special
9,Vol. 3,Filming,Late 2021


In [128]:
%ng_draw

<class 'pyvis.network.Network'> |N|=16 |E|=15

### The rendered output should look like this:

![](./graph_output.png)

# Querying and Retrieval
1. Getting a simple graph from a query

In [26]:
subgraph_retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)

In [27]:
from IPython.display import display, Markdown

nodes = subgraph_retriever.retrieve(
    "Tell me about the production of Guardians of the Galaxy 3",
)
node_text = ""
for node in nodes:
    display(Markdown(f"{node.text}"))

Guardians of the galaxy vol. 3 -> Dethroned -> Fast x

Guardians of the galaxy vol. 3 -> Directed -> Gunn

Guardians of the galaxy vol. 3 -> Projected to gross -> $110 million

Guardians of the galaxy vol. 3 -> Debuted with -> $118.4 million

Guardians of the galaxy vol. 3 -> Net profit -> $124 million

...

The making of guardians of the galaxy vol. 3 -> Released on -> Disney+

The making of guardians of the galaxy vol. 3 -> Is -> Special

Gunn chose practical effects for vol. 3 -> Said -> Beth mickle

Story of this iteration of the guardians of the galaxy -> Conclude -> Film

Directing guardians of the galaxy vol. 3 -> Discussed -> Mckay

2. Getting a NL query over the graph

In [54]:
query_engine = index.as_query_engine(
    streaming=True,
    include_text=True,  # include source text in returned nodes, default True
)

In [55]:
response = query_engine.query(
    "Tell me about the production of Guardians of the Galaxy 3 and it's relation to James Gunn?",
)
display(Markdown(response.print_response_stream()))

James Gunn was initially fired from the project due to controversial tweets, but was later rehired after a public outcry and support from the cast. He was able to complete the film after finishing his work on The Suicide Squad. The production of the film involved a mix of practical effects and virtual production technology. 


<IPython.core.display.Markdown object>

#### Check on source nodes

In [30]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='ef15fc1e-79bc-4040-94ff-5b6e2e11f460', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='51126744', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='c33130e0f3ce7b9d89c1f4373e026c0ab95a6f876e464b7ca719ba01769164a7'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='7882a1c3-d64e-425c-a1f8-10df7221c881', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='4deadad62a291998a1ae88389c2e4cee1f8c522a3a9954cbcdbdc0beffaae524'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='0e556b0f-c186-4fab-86cf-06e451666c78', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='77f855df5e91ebc6ae2cceec6d78ab72b57349378af850f6e13c4ac055fbb8d3')}, text='Here are some facts extracted from the provided text:\n\nThe making of guardians of the galaxy vol. 3 -> Released on -> Disney+\nThe making of guardians of the galaxy vol. 3 -> Is -> Sp

In [31]:
response.get_formatted_sources()

'> Source (Node id: ef15fc1e-79bc-4040-94ff-5b6e2e11f460): Here are some facts extracted from the provided text:\n\nThe making of guardians of the galaxy vol....\n\n> Source (Node id: 70f64e64-db3b-4cdf-85d4-e2b501408d05): Here are some facts extracted from the provided text:\n\nGunn chose practical effects for vol. 3 ->...\n\n> Source (Node id: 512d39cb-568a-4e92-bb8d-a30343dc340b): Here are some facts extracted from the provided text:\n\nDirecting guardians of the galaxy vol. 3 -...\n\n> Source (Node id: 073a5c35-bced-402e-aa02-13dd12d2f203): Here are some facts extracted from the provided text:\n\nStory of this iteration of the guardians o...'

3. Using a hybrid query engine

In [32]:
query_engine = index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
)

In [33]:
response = query_engine.query(
    "Tell me about the production of Guardians of the Galaxy 3?",
)
display(Markdown(f"<b>{response}</b>"))

<b>The production of Guardians of the Galaxy Vol. 3 was a complex process, with the director initially leaving the project before returning. The film was set to use virtual production technology but ultimately opted for practical effects. The film was shot at the same time as the Guardians of the Galaxy Holiday Special, which provided a tonal contrast and a break for the actors. 
</b>

In [34]:
response = await query_engine.aquery(
    "Tell me about Starlord's relationship to Groot?",
)
display(Markdown(f"<b>{response}</b>"))

<b>Star-Lord and Groot are members of the Guardians of the Galaxy and are close friends. They are often seen working together on missions. 
</b>

4. Use `TextToCypherRetriever` to generate queries

In [ ]:
from llama_index.core.indices.property_graph import TextToCypherRetriever


DEFAULT_RESPONSE_TEMPLATE = (
    "Generated Cypher query:\n{query}\n\n" "Cypher Response:\n{response}"
)
DEFAULT_ALLOWED_FIELDS = ["text", "label", "type"]

DEFAULT_TEXT_TO_CYPHER_TEMPLATE = (index.property_graph_store.text_to_cypher_template,)


cypher_retriever = TextToCypherRetriever(
    index.property_graph_store,
    # customize the LLM, defaults to Settings.llm
    llm=Settings.llm,
    # customize the text-to-cypher template.
    # Requires `schema` and `question` template args
    text_to_cypher_template=index.property_graph_store.text_to_cypher_template,
    # customize how the cypher result is inserted into
    # a text node. Requires `query` and `response` template args
    response_template=DEFAULT_RESPONSE_TEMPLATE,
    # an optional callable that can clean/verify generated cypher
    cypher_validator=None,
    # allowed fields in the resulting
    allowed_output_field=DEFAULT_ALLOWED_FIELDS,
)

In [ ]:
SCHEMA = {
    "nodes": TAGS,
    "edges": EDGES,
}

In [ ]:
nodes = cypher_retriever.retrieve(
    index.property_graph_store.text_to_cypher_template.format(
        question="Tell me about the production of Gaurdians of the Galaxy 3",
        schema=str(SCHEMA),
    )
)

In [119]:
for node in nodes:
    print(node)

Node ID: 7fc0c81e-b249-4e34-bce5-9266e785b3e9
Text: Generated Cypher query: MATCH (m:Entity__ {name: "Guardians of
the Galaxy 3"})<-[:Relation__]-(p:Props__) RETURN p   Cypher Response:
[]
Score:  1.000



# Cleanup 
%ngql
CLEAR/DROP SPACE $space_name; 